In [1]:
import numpy as np, pandas as pd, json, glob, os, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
# import shutil
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


/home/ethan/anaconda3/envs/pl_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(path="../data/qdrant_local")  # 데이터는 ./qdrant_local 폴더에 저장됨

# 2️⃣ 컬렉션 이름 및 파라미터 정의
collection_name = "medical_qa"


In [3]:
# client.delete_collection(collection_name=collection_name)

In [4]:
# client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
model = SentenceTransformer("BAAI/bge-m3", )


# RAG Description 추가

## Qdrant Collection 추가

In [5]:
content_collection = 'Major_Content'
collection_name
client.recreate_collection(
collection_name=content_collection,
vectors_config=VectorParams(size=1024, distance=Distance.COSINE))




/tmp/ipykernel_1093/552978515.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

### Collection Upload

In [ ]:
folder_path = '../../../data/1.데이터/Training/01.원천데이터/'
folder_path = '../../../medical_data/01.원천데이터/'
# 폴더 구분
points = []
for folder_ in tqdm(os.listdir(folder_path)):
    print(folder_path+folder_)
    # 파일 선택
    print(f"Folder count : {len(os.listdir(folder_path+folder_))}")
    for file_ in tqdm(os.listdir(folder_path+folder_)):
        # print(folder_path+folder_+"/"+file_)
        with open(folder_path+folder_+"/"+file_, "r", encoding="utf-8-sig") as f:
            data = json.load(f)
        data['content_prep'] = data['content'].split('. ')
        if len(data['content_prep'])<30:
            embed = model.encode(data['content_prep'])
            payload = {
                **data,  
            }

            point = PointStruct(
                id=file_.split('.')[0],
                vector=embed,
                payload=payload
            )
            points.append(point)
        elif len(data['content_prep'])<100:
            for idx, i in enumerate(range(0,len(data['content_prep']),20)):
            
                split_data = data['content_prep'][i:i+20]
                data_prep = dict(list(data.items())[:-1])
                
                embed = model.encode(split_data)
                payload = {
                    **data_prep,
                    'content_prep' : split_data
                }

                point = PointStruct(
                    id=file_.split('.')[0]+f'_split_{idx}',
                    vector=embed,
                    payload=payload
                )
                points.append(point)
        else:
            for idx, i in enumerate(range(0,len(data['content_prep']),30)):
            
                split_data = data['content_prep'][i:i+30]
                data_prep = dict(list(data.items())[:-1])
                
                embed = model.encode(split_data)
                payload = {
                    **data_prep,
                    'content_prep' : split_data
                }

                point = PointStruct(
                    id=file_.split('.')[0]+f'_split_{idx}',
                    vector=embed,
                    payload=payload
                )
                points.append(point)


  0%|          | 0/8 [00:00<?, ?it/s]

../../../medical_data/01.원천데이터/TS_국문_학술 논문 및 저널
Folder count : 9685


  0%|          | 0/8 [00:00<?, ?it/s]

{'c_id': '52949_1', 'domain': 2, 'source': 1, 'source_spec': 'Cochrane Library', 'creation_year': '2022', 'content': '상처 세척에 사용되는 물에 대한 연구에서 생리 식염수는 등장성 용액으로, 정상적인 치유 과정을 방해하지 않는다는 이유로 선호됩니다. 반면, 수돗물은 접근성이 좋고 비용 효율적이어서 커뮤니티 환경에서 흔히 사용되지만, 그 사용에 대한 논란이 여전히 존재합니다. 2021년 5월까지 Cochrane Wounds Specialized Register, CENTRAL, Ovid Medline, Ovid Embase, EBSCO CINAHL Plus 등 다양한 데이터베이스를 통해 상처 세척에 사용되는 물의 효과를 평가하기 위한 연구를 검색했습니다. 연구는 언어, 출판 날짜, 연구 환경에 제한 없이 진행되었으며, 무작위 대조 시험(RCT)을 포함했습니다. 이전 검토에 포함되었던 준 RCT는 이번 업데이트에서 제외되었습니다. 13건의 시험이 포함되었으며, 총 2504명의 참가자가 참여했습니다. 참가자들은 개방성 골절, 수술 상처, 외상성 상처, 항문 균열, 만성 상처를 경험한 환자들이었습니다. 연구는 주로 인도와 미국에서 수행되었습니다. 수돗물로 세척한 경우와 세척하지 않은 경우를 비교한 3건의 시험에서는 상처 감염에 대한 데이터가 보고되지 않았습니다. 생리 식염수와 수돗물을 비교한 8건의 시험에서는 수돗물의 효과가 불확실하다는 결과가 나왔습니다(RR 0.84, 95% CI 0.59~1.19). 개방성 골절의 경우, 증류수와 생리 식염수를 비교한 2건의 시험에서도 감염된 골절 수에 대한 효과가 불확실했습니다(RR 0.70, 95% CI 0.45~1.09). 냉각된 끓인 물과 생리 식염수를 비교한 시험에서도 감염된 골절 수에 대한 효과가 불확실했습니다(RR 0.83, 95% CI 0.37~1.87). 상처 치유와 관련해서는, 수돗물로 세척한 경우와 세척하지 않은 경우를

In [29]:
# folder_path = '../../../data/1.데이터/Training/01.원천데이터/'
# # 폴더 구분
# points = []
# for folder_ in tqdm(os.listdir(folder_path)):
#     print(folder_path+folder_)
#     # 파일 선택
#     print(f"Folder count : {len(os.listdir(folder_path+folder_))}")
#     for file_ in tqdm(os.listdir(folder_path+folder_)):
#         # print(folder_path+folder_+"/"+file_)
#         with open(folder_path+folder_+"/"+file_, "r", encoding="utf-8-sig") as f:
#             data = json.load(f)
#         data['content_prep'] = data['content'].split('. ')

        
#         embed = model.encode(data['content_prep'])
#         payload = {
#             **data,  
#         }

#         point = PointStruct(
#             id=file_.split('.')[0],
#             vector=embed,
#             payload=payload
#         )
#         points.append(point)    
        
    
#     #     break
#     # break
    

# Question

In [7]:
query_data = {
    "qa_id": 4286,
    "domain": 17,
    "q_type": 1,
    "question": "65세 남자가 심질환으로 현재 aspirin을 복용 중이다. 이 환자에서 aspirin 복용으로 인해 나타날 수 있는 소견으로 가장 적절한 것은?  \n1) PTT 증가  \n2) TT 증가  \n3) 혈소판 감소  \n4) PT 증가  \n5) BT 증가",
    "answer": "5) BT 증가"
}

# domain, q_type 기준으로 필터 설정
filters = models.Filter(
    must=[
        models.FieldCondition(
            key="domain",
            match=models.MatchValue(value=query_data["domain"])
        ),
        models.FieldCondition(
            key="q_type",
            match=models.MatchValue(value=query_data["q_type"])
        ),
    ]
)

In [8]:
query_vector = model.encode(query_data["question"]).tolist()

In [9]:
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    query_filter=filters
)

/tmp/ipykernel_1874709/1981553409.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [10]:
for i, r in enumerate(results):
    print(f"{i+1}. Score: {r.score:.4f}")
    print(f"Question: {r.payload['question']}")
    print(f"Answer: {r.payload['answer']}\n")


1. Score: 0.7091
Question: 65세 남성이 급성 심근경색으로 응급실에 내원하였다. 이 환자의 초기 치료로 권장되는 aspirin의 최소 부하 용량은 얼마인가?  
1) 50 mg  
2) 75 mg  
3) 100 mg  
4) 160~325 mg  
5) 500 mg
Answer: 4) 160~325 mg

2. Score: 0.7024
Question: 65세 남성이 관상동맥 스텐트 시술 후 항혈전 치료를 받고 있다. 티클로피딘과 아스피린을 병용 투여할 경우, 다음 중 주요한 치료 효과로 기대되는 것은 무엇인가?  
1) 총 사망률 감소  
2) 혈전성 혈소판 감소성 자반증(TTP) 발생 감소  
3) 비치명적 심근경색 발생 감소  
4) 백혈구 감소증 발생 감소  
5) 경미한 출혈 증가
Answer: 3) 비치명적 심근경색 발생 감소

3. Score: 0.6967
Question: 60세 남성이 갑작스러운 심한 가슴 통증으로 응급실에 내원하였습니다. 심전도에서 ST 분절 상승이 관찰되었고, 혈액 검사에서 트로포닌 수치가 상승하였습니다. 이 환자에서 가장 우선적으로 시행해야 할 치료는 무엇인가요?

1) 심장 초음파  
2) 혈전 용해제 투여  
3) 응급 관상동맥 중재술(PCI) 
4) 니트로글리세린 설하정  
5) 아스피린 복용
Answer: 3) 응급 관상동맥 중재술(PCI)



In [12]:
# # 3️⃣ 임베딩 모델 로드 (bge-m3)
# model = SentenceTransformer("BAAI/bge-m3")

# # 5️⃣ question 임베딩 생성
# # embedding = model.encode(data["question"]).tolist()

# # 6️⃣ Qdrant에 데이터 업로드
# client.upsert(
#     collection_name=collection_name,
#     points=[
#         PointStruct(
#             id=data["qa_id"],
#             vector=embedding,
#             payload={
#                 "domain": data["domain"],
#                 "q_type": data["q_type"],
#                 "question": data["question"],
#                 "answer": data["answer"],
#             },
#         )
#     ],
# )

# print("✅ 데이터 업로드 완료!")

In [13]:
# if collection_name not in [c.name for c in client.get_collections().collections]:
#     client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
#     print(f"✅ 컬렉션 '{collection_name}' 생성 완료")
# else:
#     print(f"ℹ️ 컬렉션 '{collection_name}' 이미 존재")